In [35]:
# pip install zipcodes
import pandas as pd
import numpy as np
import json
import zipcodes
# file = pd.read_csv('loan.csv')
# df = pd.DataFrame(file)

In [36]:
file = pd.read_csv('loan.csv', usecols = [ 'addr_state','zip_code', 'loan_amnt','annual_inc'])
loan_amnt = pd.DataFrame(file)

In [37]:
loan_amnt_zip = loan_amnt.groupby(['addr_state','zip_code']).agg({'loan_amnt': np.sum, 'annual_inc': np.mean}).reset_index()
loan_amnt_zip.head()

,addr_state,zip_code,loan_amnt,annual_inc
0,AK,009xx,21625,75000.0
1,AK,025xx,30000,100000.0
2,AK,090xx,35000,109000.0
3,AK,112xx,8000,68000.0
4,AK,300xx,5400,55000.0


In [56]:

def convertzip_county(zip):
    zip = zip[0:3]
    county = zipcodes.similar_to(zip)
    if county:
        county = county[0]['state']
#         county = county.rsplit(' ', 1)[0]
    else: county = ""
    return county


loan_amnt_zip['zip_state'] = loan_amnt_zip['zip_code'].apply(convertzip_county)
loan_amnt_zip.head()

,addr_state,zip_code,loan_amnt,annual_inc,zip_state
0,AK,009xx,21625,75000.0,PR
1,AK,025xx,30000,100000.0,MA
2,AK,090xx,35000,109000.0,AE
3,AK,112xx,8000,68000.0,NY
4,AK,300xx,5400,55000.0,GA


In [57]:
loan_amnt_zip_new = loan_amnt_zip.loc[loan_amnt_zip.addr_state == loan_amnt_zip.zip_state]

In [58]:
del loan_amnt_zip_new['zip_state']
loan_amnt_zip_new.head()

,addr_state,zip_code,loan_amnt,annual_inc
34,AK,995xx,42938975,80652.832982
35,AK,996xx,21832325,83295.471302
36,AK,997xx,16265025,77775.333875
37,AK,998xx,6399125,72974.729271
38,AK,999xx,2451825,68294.797303


In [59]:
# def convertzip_county(zip):
#     zip = zip[0:3]
#     county = zipcodes.similar_to(zip)
#     if county:
#         county = county[0]['county']
#         county = county.rsplit(' ', 1)[0]
#     else: county = ""
#     return county


# loan_amnt_zip_new['county'] = loan_amnt_zip_new['zip_code'].apply(convertzip_county)
# loan_amnt_zip_new.head()

In [60]:
df = pd.DataFrame(columns=['zip_code', 'county']) 
for zip in loan_amnt_zip_new['zip_code']:
    zip1 = zip[0:3]
    county = zipcodes.similar_to(zip1)
    res = [sub['county'].rsplit(' ', 1)[0] for sub in county ] 
    res = list(set(res))  
    for k in res:
        if k:
            df = df.append({'zip_code' : zip , 'county' : k} , ignore_index=True)
df

,zip_code,county
0,995xx,Anchorage
1,995xx,Aleutians East
2,995xx,Valdez-Cordova Census
3,995xx,Lake and Peninsula
4,995xx,Kodiak Island
5,995xx,Wade Hampton Census
6,995xx,Yukon-Koyukuk Census
7,995xx,Bethel Census
8,995xx,Kenai Peninsula
9,995xx,North Slope


In [61]:
loan_amnt_zip_new = pd.merge(df, loan_amnt_zip_new, on='zip_code')

In [62]:
loan_amnt_state = loan_amnt.groupby(['addr_state','zip_code']).agg({'loan_amnt': np.sum, 'annual_inc': np.mean}).reset_index()
loan_amnt_state = loan_amnt_state.groupby(['addr_state']).agg({'loan_amnt': np.mean, 'annual_inc': np.mean}).reset_index()
loan_amnt_state.head()

,addr_state,loan_amnt,annual_inc
0,AK,2.318459e+06,80308.593968
1,AL,4.355366e+06,71642.322850
2,AR,3.760164e+06,73852.744896
3,AZ,8.976622e+06,73852.728083
4,CA,2.278901e+07,82726.682685


In [63]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

loan_amnt_zip_new['state'] = loan_amnt_zip_new['addr_state'].apply(lambda x: state_codes[x])
loan_amnt_state['state'] = loan_amnt_state['addr_state'].apply(lambda x: state_codes[x])

In [64]:
loan_amnt_state=loan_amnt_state.rename(columns = {'addr_state':'id'})


loan_amnt_zip_new=loan_amnt_zip_new.rename(columns = {'addr_state':'id'})


loan_amnt_state = json.loads(loan_amnt_state.to_json(orient='records'))
loan_amnt_zip_new = json.loads(loan_amnt_zip_new.to_json(orient='records'))

In [65]:
loan_amnt_state

[{'id': 'AK',
  'loan_amnt': 2318458.9743589745,
  'annual_inc': 80308.5939675003,
  'state': '02'},
 {'id': 'AL',
  'loan_amnt': 4355365.760869565,
  'annual_inc': 71642.3228502759,
  'state': '01'},
 {'id': 'AR',
  'loan_amnt': 3760164.0625,
  'annual_inc': 73852.7448958075,
  'state': '05'},
 {'id': 'AZ',
  'loan_amnt': 8976622.126436781,
  'annual_inc': 73852.7280833603,
  'state': '04'},
 {'id': 'CA',
  'loan_amnt': 22789005.213270143,
  'annual_inc': 82726.6826846846,
  'state': '06'},
 {'id': 'CO',
  'loan_amnt': 8176676.685393258,
  'annual_inc': 83248.50853261,
  'state': '08'},
 {'id': 'CT',
  'loan_amnt': 8555577.34375,
  'annual_inc': 80668.8020259512,
  'state': '09'},
 {'id': 'DC',
  'loan_amnt': 2289385.135135135,
  'annual_inc': 78400.9106642555,
  'state': '11'},
 {'id': 'DE',
  'loan_amnt': 4822548.75,
  'annual_inc': 71597.3817956805,
  'state': '10'},
 {'id': 'FL',
  'loan_amnt': 15763746.621621622,
  'annual_inc': 77841.0770031554,
  'state': '12'},
 {'id': 'GA',
 

In [66]:
loan_amnt_zip_new

[{'zip_code': '995xx',
  'county': 'Anchorage',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.8329819639,
  'state': '02'},
 {'zip_code': '995xx',
  'county': 'Aleutians East',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.8329819639,
  'state': '02'},
 {'zip_code': '995xx',
  'county': 'Valdez-Cordova Census',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.8329819639,
  'state': '02'},
 {'zip_code': '995xx',
  'county': 'Lake and Peninsula',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.8329819639,
  'state': '02'},
 {'zip_code': '995xx',
  'county': 'Kodiak Island',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.8329819639,
  'state': '02'},
 {'zip_code': '995xx',
  'county': 'Wade Hampton Census',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.8329819639,
  'state': '02'},
 {'zip_code': '995xx',
  'county': 'Yukon-Koyukuk Census',
  'id': 'AK',
  'loan_amnt': 42938975,
  'annual_inc': 80652.832981

In [67]:
with open('data.txt', 'w') as outfile:
    json.dump(loan_amnt_zip_new, outfile)

In [44]:
a = zipcodes.similar_to('996')

In [45]:
res = [ sub['county'].rsplit(' ', 1)[0] for sub in a ] 

In [46]:
list(set(res))  

['',
 'Aleutians East',
 'Bristol Bay',
 'Valdez-Cordova Census',
 'Lake and Peninsula',
 'Wade Hampton Census',
 'Kodiak Island',
 'Yukon-Koyukuk Census',
 'Nome Census',
 'Yakutat City and',
 'Bethel Census',
 'Kenai Peninsula',
 'Matanuska-Susitna',
 'Aleutians West Census',
 'Municipality of',
 'Dillingham Census']